In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle

from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split

In [3]:
#tail -n +2 train.csv | split -l 150000


In [4]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [ ]:
TIMESTEPS=10000


import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

split_chunks = np.array_split(splits,mp.cpu_count())

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS]) for chunk in split_chunks]
    pool.close()
    pool.join()

In [ ]:
training = np.empty((len(splits),TIMESTEPS,1),dtype=float)
targets = np.empty((len(splits),1),dtype=float)
i=0
for r in res:
    for df in r.get():
        training[i] = df.loc[:,df.columns != 'time_to_failure']
        #training[i] = df[['acoustic_data','rolling_100']]
        targets[i] = np.array([df['time_to_failure'].unique()])
        i+=1

In [ ]:
res[0].get()[0].head()

In [ ]:
res[0].get()[0][['acoustic_data', 'rolling_10', 'rolling_50', 'rolling_100', 'min_50',
       'max_50', 'std_50']].shape

In [ ]:

train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=0.1, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


dropout=0.2
BATCH_SIZE=64


my_model = Sequential()

my_model.add(CuDNNLSTM(#use_bias = True,unit_forget_bias=True,\
                  units = 4,\
                  #stateful=True,
                    #batch_input_shape=(BATCH_SIZE,TIMESTEPS,2),\
                  #dropout=dropout,recurrent_dropout=dropout,
                    #return_sequences=True
                  ))

#my_model.add(BatchNormalization())
#my_model.add(CuDNNLSTM(#use_bias = True,unit_forget_bias=True,\
                 # units = 8,\
                  #stateful=True,
                    #batch_input_shape=(BATCH_SIZE,TIMESTEPS,2),\
                  #dropout=dropout,recurrent_dropout=dropout,
                 # ))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
#my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=0)
]

history = my_model.fit(train_data, y_train, batch_size=BATCH_SIZE, epochs=100, #shuffle=False,
                      validation_data=(val_data,y_val), callbacks=callbacks
                    )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

In [ ]:
next(my_training_batch_generator)

In [5]:

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_g

split_chunks = np.array_split(splits,mp.cpu_count())

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_g,args=[chunk]) for chunk in split_chunks]
    pool.close()
    pool.join()

xzfgi


In [ ]:
res[0].get()[0]

In [9]:
training = np.empty((100,150000))
i=0
for r in res:
    for df in r.get():
        #print(df.shape)
        if i < 100:
            training[i] = df
        #training[i] = df[['acoustic_data','rolling_100']]
        i+=1

In [10]:

len(training)

100

In [12]:
from keras.layers import Input, Dense
from keras.models import Model


train_data, val_data, y_train, y_val = train_test_split(training, training, test_size=0.1, random_state=42)

Using TensorFlow backend.


In [ ]:


input_dim = Input(shape = (150000, ))

# Encoder Layers
#encoded0 = Dense(50000, activation = 'relu')(input_dim)
encoded1 = Dense(10000, activation = 'relu')(input_dim)
encoded2 = Dense(5000, activation = 'relu')(encoded1)
encoded3 = Dense(2500, activation = 'relu')(encoded2)
encoded4 = Dense(1250, activation = 'relu')(encoded3)
encoded5 = Dense(750, activation = 'relu')(encoded4)
encoded6 = Dense(500, activation = 'relu')(encoded5)

# Decoder Layers
decoded0 = Dense(500, activation = 'relu')(encoded6)
decoded1 = Dense(750, activation = 'relu')(decoded0)
decoded2 = Dense(1250, activation = 'relu')(decoded1)
decoded3 = Dense(2500, activation = 'relu')(decoded2)
decoded4 = Dense(5000, activation = 'relu')(decoded3)
decoded5 = Dense(10000, activation = 'relu')(decoded4)
#decoded6 = Dense(50000, activation = 'relu')(decoded5)
decoded7 = Dense(150000)(decoded5)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded7)

# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'mse')

autoencoder.fit(train_data, train_data, nb_epoch = 10, batch_size = 1, shuffle = False, validation_data = (val_data, val_data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 90 samples, validate on 10 samples
Epoch 1/10
